# Yahoo!ショッピングのカテゴリーＩＤ一覧を取得する

In [7]:
import requests
import json
import time
import csv

# end point
url_cat = 'https://shopping.yahooapis.jp/ShoppingWebService/V1/json/categorySearch'
with open('yahooappid.txt', 'r') as f:
    appid = f.read()

all_categories_file = './all_categories.csv'

# APIリクエスト呼び出し用関数
def r_get(url, dct):
    time.sleep(1)
    return requests.get(url, params=dct)

# カテゴリ取得用関数
def get_cats(cat_id):
    try:
        result = r_get(url_cat, {'appid': appid, 'category_id': cat_id})
        cats = result.json()['ResultSet']['0']['Result']['Categories']['Children']
        for i, cat in cat.items():
            if not i == '_container':
                yield cat['Id'], {'short': cat['Title']['Short'],
                                  'medium': cat['Title']['Medium'], 
                                  'long': cat['Title']['Long']}
    except:
        pass
    

# カテゴリー一覧ＣＳＶファイルの生成

In [9]:
output_buffer = [['カテゴリコードlv1', 'カテゴリコードlv2', 'カテゴリコードlv3',
                 'カテゴリ名lv1', 'カテゴリ名lv2', 'カテゴリ名lv3', 'カテゴリ名lv3_long']]

with open(all_categories_file, 'w') as f:
    writer = csv.writer(f, lineterminator='\n')
    writer.writerows(output_buffer)
    output_buffer = []

for id1, title1 in get_cats(1):
    print('カテゴリレベル１: ', title1['short'])
    try:
        for id2, title2 in get_cats(id1):
            for id3, title3 in get_cats(id2):
                wk = [id1, id2, id3, title1['short'], title2['short'], title3['short'], title3['long']]
                output_buffer.appned(wk)
                with open(all_categories_file, 'a') as f:
                    writer = csv.writer(f, lineterminator='\n')
                    writer.writerows(output_buffer)
                    output_buffer = []
    except KeyError:
        continue
                